# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846762251120                   -0.70    4.2         
  2   -7.852288787563       -2.26       -1.53    1.0   28.2ms
  3   -7.852613914200       -3.49       -2.55    1.8   32.2ms
  4   -7.852645952319       -4.49       -2.88    2.5   38.2ms
  5   -7.852646502514       -6.26       -3.19    1.0   27.8ms
  6   -7.852646679782       -6.75       -4.05    1.2   29.3ms
  7   -7.852646686630       -8.16       -5.13    1.8   35.0ms
  8   -7.852646686726      -10.02       -5.53    2.0   35.5ms
  9   -7.852646686729      -11.47       -5.84    1.5   31.1ms
 10   -7.852646686730      -12.07       -6.76    1.0   29.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846793857432                   -0.70           4.5         
  2   -7.852523722202       -2.24       -1.64   0.80    2.0    309ms
  3   -7.852634489083       -3.96       -2.73   0.80    1.0   26.2ms
  4   -7.852646447505       -4.92       -3.22   0.80    2.2   35.6ms
  5   -7.852646666012       -6.66       -4.03   0.80    1.0   25.9ms
  6   -7.852646686353       -7.69       -4.69   0.80    1.8   31.6ms
  7   -7.852646686706       -9.45       -5.44   0.80    1.5   29.6ms
  8   -7.852646686729      -10.63       -6.65   0.80    2.0   32.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.407181e+01     3.698735e+00
 * time: 0.4698197841644287
     1     1.221729e+00     2.144132e+00
 * time: 0.6976048946380615
     2    -1.343581e+00     2.515894e+00
 * time: 0.7256410121917725
     3    -3.492710e+00     2.174440e+00
 * time: 0.7654728889465332
     4    -4.510256e+00     2.214217e+00
 * time: 0.8042428493499756
     5    -6.219041e+00     1.388732e+00
 * time: 0.8431007862091064
     6    -7.005222e+00     7.358489e-01
 * time: 0.8822929859161377
     7    -7.408341e+00     5.269211e-01
 * time: 0.9092328548431396
     8    -7.641025e+00     1.641274e-01
 * time: 0.9362127780914307
     9    -7.764646e+00     1.896864e-01
 * time: 0.9633979797363281
    10    -7.813177e+00     8.247559e-02
 * time: 0.990562915802002
    11    -7.839890e+00     5.958862e-02
 * time: 1.0176868438720703
    12    -7.846816e+00     3.340002e-02
 * time: 1.0447659492492676
    13    -7.849859e+00     3.124541e-02
 * time: 1.07235789299

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846664746137                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645923705                   -1.64         
  2   -7.852646686730       -6.12       -3.71    1.94s
  3   -7.852646686730      -13.37       -7.26    238ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.0776215597058799e-7
|ρ_newton - ρ_scfv| = 3.839535111967938e-7
|ρ_newton - ρ_dm|   = 1.2339789980750775e-9
